<a href="https://colab.research.google.com/github/Astle-sudo/3D/blob/main/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries**

In [7]:
import numpy
import tensorflow as flow
import os
import zipfile
import matplotlib.pyplot as plt

# **Simple Linear Network**

In [3]:
# model = flow.keras.Sequential([flow.keras.layers.Dense(units=1, input_shape=[1])])
# model.compile(optimizer='sgd', loss='mean_squared_error')

# x = numpy.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
# y = numpy.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# model.fit(x, y, epochs=500, verbose=0)

x = flow.range(0,10)

y = 2 * x

print(y)

tf.Tensor([ 0  2  4  6  8 10 12 14 16 18], shape=(10,), dtype=int32)


# **Convolutional Neural Network**

In [ ]:
# !wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
# !wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
# local_zip = './horse-or-human.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('./horse-or-human')

# local_zip = './validation-horse-or-human.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('./validation-horse-or-human')

local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

In [ ]:
# train_horse_dir = os.path.join('./horse-or-human/horses')
# train_human_dir = os.path.join('./horse-or-human/humans')
# validation_horse_dir = os.path.join('./validation-horse-or-human/horses')
# validation_human_dir = os.path.join('./validation-horse-or-human/humans')

# train_horse_names = os.listdir(train_horse_dir)
# train_human_names = os.listdir(train_human_dir)
# validation_horse_hames = os.listdir(validation_horse_dir)
# validation_human_names = os.listdir(validation_human_dir)

base_dir = 'cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

In [ ]:
# fmist = flow.keras.datasets.fashion_mnist
# (training_images, training_lables), (test_images, test_lables) = fmist.load_data()
# training_images = training_images/ 255.0
# test_images = test_images/ 255.0

In [ ]:
class myCallBack (flow.keras.callbacks.Callback):
  def on_epoch_end (self, epoch, logs={}):
    if logs.get('accuracy') is not None and logs.get('accuracy') > 0.70:
      self.model.stop_training = True

In [ ]:
model = flow.keras.models.Sequential([
    flow.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(150, 150, 3)),
    flow.keras.layers.MaxPooling2D(2, 2),
    flow.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    flow.keras.layers.MaxPooling2D(2,2),
    flow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    flow.keras.layers.MaxPooling2D(2,2),
    flow.keras.layers.Flatten(),
    flow.keras.layers.Dense(512, activation='relu'),
    flow.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer = flow.keras.optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# model.fit (training_images, training_lables, epochs=5, callbacks=[CallBack ()], verbose=0)

In [ ]:
model.summary()

In [ ]:
train_datagen = flow.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
  )
validation_datagen = flow.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, # './horse-or-human/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

val_generator = validation_datagen.flow_from_directory(
    validation_dir, # './validation-horse-or-human/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

In [ ]:
# history = model.fit(train_generator,
#                     steps_per_epoch=100,
#                     epochs=15, verbose=2,
#                     validation_data=val_generator,
#                     validation_steps=50,
#                   )

In [ ]:
# acc = history.history['accuracy']
# loss = history.history['loss']

# plt.plot(range(1, 16), acc)
# # plt.plot(range(1, 16), loss)
# plt.show()

# **Transfer Learning**

In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = flow.keras.applications.inception_v3.InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
x = flow.keras.layers.Flatten()(last_output)
x = flow.keras.layers.Dense(1024, activation='relu')(x)
x = flow.keras.layers.Dropout(0.2)(x)
x = flow.keras.layers.Dense(1, activation='sigmoid')(x)
model = flow.keras.Model(pre_trained_model.input, x)
model.compile(optimizer = flow.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# history = model.fit(train_generator,validation_data = validation_generator,steps_per_epoch = 100,epochs = 20,validation_steps = 50,verbose = 2)

# **Recurrent Neural Networks**

In [1]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2023-11-28 21:53:57--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c06::cf, 2607:f8b0:4023:c0d::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  --.-KB/s    in 0.08s   

2023-11-28 21:53:57 (66.5 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [3]:
import json

with open("./sarcasm.json", 'r') as f:
    datastore = json.load(f)

In [4]:
sentences = list()
labels = list()
urls = list()

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [19]:
vocab_size = 10000
max_length = 40
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = flow.keras.preprocessing.text.Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded = flow.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
labels = numpy.array(labels).reshape(len(labels),1)

print(padded.shape, padded[0].shape, padded[0])

In [21]:
model = flow.keras.Sequential([
    flow.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    flow.keras.layers.GlobalAveragePooling1D(),
    flow.keras.layers.Dense(24, activation='relu'),
    flow.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
num_epochs = 10
history = model.fit(padded, labels, epochs=num_epochs, verbose=2)

Epoch 1/10
835/835 - 20s - loss: 0.5285 - accuracy: 0.7240 - 20s/epoch - 24ms/step
Epoch 2/10
835/835 - 5s - loss: 0.2967 - accuracy: 0.8791 - 5s/epoch - 6ms/step
Epoch 3/10
835/835 - 3s - loss: 0.2355 - accuracy: 0.9073 - 3s/epoch - 4ms/step
Epoch 4/10
835/835 - 4s - loss: 0.1986 - accuracy: 0.9234 - 4s/epoch - 5ms/step
Epoch 5/10
835/835 - 4s - loss: 0.1729 - accuracy: 0.9335 - 4s/epoch - 4ms/step
Epoch 6/10
835/835 - 4s - loss: 0.1540 - accuracy: 0.9423 - 4s/epoch - 4ms/step
Epoch 7/10
835/835 - 3s - loss: 0.1388 - accuracy: 0.9483 - 3s/epoch - 4ms/step
Epoch 8/10
835/835 - 3s - loss: 0.1268 - accuracy: 0.9552 - 3s/epoch - 3ms/step
Epoch 9/10
835/835 - 3s - loss: 0.1157 - accuracy: 0.9582 - 3s/epoch - 4ms/step
Epoch 10/10
835/835 - 4s - loss: 0.1067 - accuracy: 0.9617 - 4s/epoch - 4ms/step


# **LSTMs**

In [ ]:
!gdown --id 15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT

In [ ]:
data = open('./irish-lyrics-eof.txt').read()
corpus = data.lower().split("\n")

In [ ]:
tokenizer = flow.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = flow.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = flow.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
embedding_dim = 100
lstm_units = 150
learning_rate = 0.01

print(xs[0])

model = flow.keras.Sequential([
          flow.keras.layers.Embedding(total_words, embedding_dim, input_length=max_sequence_len-1),
          flow.keras.layers.Bidirectional(flow.keras.layers.LSTM(lstm_units)),
          flow.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=flow.keras.optimizers.Adam(learning_rate=learning_rate),
    metrics=['accuracy']
    )
model.summary()

In [ ]:
epochs = 10
history = model.fit(xs, ys, epochs=epochs)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

plot_graphs(history, 'accuracy')

In [ ]:
seed_text = "help me obi-wan kinobi youre my only hope"
next_words = 100

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = flow.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	probabilities = model.predict(token_list, verbose=0)
	predicted = numpy.argmax(probabilities, axis=-1)[0]
	if predicted != 0:
		output_word = tokenizer.index_word[predicted]
		seed_text += " " + output_word
print(seed_text)

# **Time Series**

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.figure(figsize=(10, 6))
    if type(series) is tuple:
      for series_num in series:
        plt.plot(time[start:end], series_num[start:end], format)
    else:
      plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
    plt.show()

In [ ]:
def trend(time, slope=0):
    series = slope * time
    return series

def seasonal_pattern(season_time):
    data_pattern = numpy.where(season_time < 0.4,numpy.cos(season_time * 2 * numpy.pi),1 / numpy.exp(3 * season_time))
    return data_pattern

def seasonality(time, period, amplitude=1, phase=0):
    season_time = ((time + phase) % period) / period
    data_pattern = amplitude * seasonal_pattern(season_time)
    return data_pattern

def noise(time, noise_level=1, seed=None):
    rnd = numpy.random.RandomState(seed)
    noise = rnd.randn(len(time)) * noise_level
    return noise

In [ ]:
time = numpy.arange(4 * 365 + 1, dtype="float32")
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
series += noise(time, noise_level, seed=42)
plot_series(time, series)

In [ ]:
split_time = 1000

time_train = time[:split_time]
x_train = series[:split_time]

time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
naive_forecast = series[split_time - 1:-1]
time_step = 100

In [ ]:
def moving_average_forecast(series, window_size):
    forecast = []
    for time in range(len(series) - window_size):
      forecast.append(series[time:time + window_size].mean())
    forecast = numpy.array(forecast)

    return forecast

In [ ]:
moving_avg = moving_average_forecast(series, 30)[split_time - 30:]
plot_series(time_valid, (x_valid, moving_avg))
diff_series = (series[365:] - series[:-365])
diff_time = time[365:]
plot_series(diff_time, diff_series)
diff_moving_avg = moving_average_forecast(diff_series, 30)
diff_moving_avg = diff_moving_avg[split_time - 365 - 30:]
diff_series = diff_series[split_time - 365:]
plot_series(time_valid, (diff_series, diff_moving_avg))
diff_moving_avg_plus_past = series[split_time - 365:-365] + diff_moving_avg
plot_series(time_valid, (x_valid, diff_moving_avg_plus_past))
diff_moving_avg_plus_smooth_past = moving_average_forecast(series[split_time - 370:-359], 11) + diff_moving_avg
plot_series(time_valid, (x_valid, diff_moving_avg_plus_smooth_past))

In [ ]:
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = flow.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
# model_baseline = flow.keras.models.Sequential([
#     flow.keras.layers.Dense(10, input_shape=[window_size], activation="relu"),
#     flow.keras.layers.Dense(10, activation="relu"),
#     flow.keras.layers.Dense(1)
# ])

# lr_schedule = flow.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))

# model_baseline.summary()
# model_baseline.compile(loss="mse", optimizer=flow.keras.optimizers.SGD(learning_rate=7e-6, momentum=0.9))
# model_baseline.fit(dataset,epochs=10)

In [ ]:
flow.keras.backend.clear_session()
model_baseline = flow.keras.models.Sequential([
    flow.keras.layers.Lambda(lambda x: flow.expand_dims(x, axis=-1), input_shape=[None]),
    flow.keras.layers.Bidirectional(flow.keras.layers.LSTM(32)),
    flow.keras.layers.Dense(1),
    flow.keras.layers.Lambda(lambda x: x * 100.0)
])

model_baseline.compile(loss='mse', optimizer=flow.keras.optimizers.SGD(learning_rate=5e-6, momentum=0.9))
model_baseline.fit(dataset, epochs=10, verbose=0)

In [ ]:
forecast = []
forecast_series = series[split_time - window_size:]
for time in range(len(forecast_series) - window_size):
  forecast.append(model_baseline.predict(forecast_series[time:time + window_size][numpy.newaxis]))
results = numpy.array(forecast).squeeze()
plot_series(time_valid, (x_valid, results))

In [ ]:
print(flow.keras.metrics.mean_squared_error(x_valid, results).numpy())
print(flow.keras.metrics.mean_absolute_error(x_valid, results).numpy())

In [1]:
# window_size = 60
# batch_size = 32
# trained_set = dataset

# model = flow.keras.models.Sequential([
#     flow.keras.layers.Conv1D(filters=32, kernel_size=5, padding='casual', strides=1, activation='relu', input_shape=[None, 1]),
#     flow.keras.layers.LSTM(32, return_sequences=True),
#     flow.keras.layers.LSTM(32),
#     flow.keras.layers.Dense(30, activation='relu'),
#     flow.keras.layers.Dense(10, activation='relu'),
#     flow.keras.layers.Dense(1),
#     flow.keras.layers.Lambda(lambda x: x * 400.0)
# ])

# lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-8, momentum=0.9)
# model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)
# history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])